In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 载入数据
mnist = input_data.read_data_sets('D:/kaggle-10-most-popular-dataset/MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/kaggle-10-most-popular-dataset/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/kaggle-10-most-popular-dataset/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting D:/kaggle-10-most-popular-dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting D:/kaggle-10-most-popular-dataset/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 批次的大小
batch_size = 128
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)                                         # dropout率
lr = tf.Variable(1,dtype = tf.float32)                                         # 学习率  

                                                                               # 创建神经网络
W1 = tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))
b1 = tf.Variable(tf.zeros([1, 2000]))
                                                                               # 激活层
layer1 = tf.nn.relu(tf.matmul(x,W1) + b1)
                                                                               # drop层
layer1 = tf.nn.dropout(layer1,keep_prob=keep_prob)

                                                                               # 第二层
W2 = tf.Variable(tf.truncated_normal([2000,500],stddev=0.1))
b2 = tf.Variable(tf.zeros([1, 500]))
layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
layer2 = tf.nn.dropout(layer2,keep_prob=keep_prob)

                                                                              # 输出层
W3 = tf.Variable(tf.truncated_normal([500,10],stddev=0.1))
b3 = tf.Variable(tf.zeros([1,10]))
prediction = tf.nn.softmax(tf.matmul(layer2,W3) + b3)                         # 预测值为one-hot 编码
#prediction = tf.matmul(layer2,W3) + b3


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))  # 交叉熵

 # 梯度下降法
# train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
train_step = tf.train.AdadeltaOptimizer(lr).minimize(loss)                                   # Adadelta 优化           
# train_step = tf.train.AdamOptimizer().minimize(loss)  
# train_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss) 
# train_step = tf.train.AdagradOptimizer(learning_rate=1).minimize(loss) 


                                                                                            # 初始化变量
init = tf.global_variables_initializer()

prediction_2 = tf.nn.softmax(prediction)
                                                                                           # 得到一个布尔型列表，存放结果是否正确
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction_2,1))                   # argmax 返回一维张量中最大值索引  （相当于应用于one-hot编码的判别是否正确的函数）

                                                                                           # 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))                          # tf.cast把布尔值转换为浮点型，然后求平均数

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):                                                                 # 1个epoch等于使用训练集中的全部样本训练一次； 
        sess.run(tf.assign(lr, 1*(0.98**epoch)))                                            # lr为 learning rate 初始值为0.001 迭代一次更新一次学习率  ,学习率会逐渐降低
        for batch in range(n_batch):                                                        # batch_size 批大小。在深度学习中，一般采用SGD训练，即每次训练在训练集中取batchsize个样本训练；
                                                                                            # 获得批次数据
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.8})
        learning_rate = sess.run(lr)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0} )
        print("Iter " + str(epoch) + " Testing Accuracy: " + str(acc)+ ",  lr: "+ str(learning_rate))

Iter 0 Testing Accuracy: 0.9431,  lr: 1.0
Iter 1 Testing Accuracy: 0.9535,  lr: 0.98
Iter 2 Testing Accuracy: 0.9606,  lr: 0.9604
Iter 3 Testing Accuracy: 0.967,  lr: 0.941192
Iter 4 Testing Accuracy: 0.9675,  lr: 0.92236817
Iter 5 Testing Accuracy: 0.9721,  lr: 0.90392077
Iter 6 Testing Accuracy: 0.973,  lr: 0.8858424
Iter 7 Testing Accuracy: 0.973,  lr: 0.86812556
Iter 8 Testing Accuracy: 0.977,  lr: 0.850763
Iter 9 Testing Accuracy: 0.9769,  lr: 0.83374774
